# Explore one prompt at a time
This notebook is meant to explore the process of collecting concepts. The user can vary the prompt, the layers and the number of images to use for the prompt. 

In [ ]:
import os
import time 

import argparse
import asyncio
import csv
import numpy as np
import random

import torch 

import openai
import base64

from utils import *

In [ ]:
np.random.seed(0)
random.seed(0)

In [ ]:
client = openai.OpenAI(
    api_key="INSERT YOUR KEY HERE")

In [ ]:
prompts = {'0': ( "You are a visual concept tagger and compressor. Given one or a group of images that activate the same neuron, your job is to extract a single, specific visual concept that appears prominently in all of them. This concept must be: - Concrete (e.g., 'red double-decker bus', not 'transportation') - Visual (can be drawn by an image generation model like SDXL) - Consistent (appears in every image). Avoid vague or abstract themes like 'diversity', 'context', or 'life'. For example, do not use 'animal' but specify what kind of animal or characteristics of the animal. Focus on visible entities (objects, creatures, materials, or actions). Output only the final concept that occurs in all images, no other text. Always output only a visual concept, if there is not a clear one, get the most likely. Do not state that there is no clear visual concept, just output the concept" )}

IMAGE_NUMBER = [1,5,10,15,20]

LAYERS = [1,2,3,4]

In [ ]:
def prompt_single(top_x_indices, prompt=None):
    if prompt is None:
        prompt = (
            "Here are several images that activate the same neuron. "
            "Identify a concrete, tangible visual concept that appears in all of them. "
            "Be specific—focus on objects, environments, or actions that visually dominate the images. "
            "The concept should be easily representable in an image generation model like SDXL. "
            "Output only the final concept, no additional text."
        )

    encoded_images = get_image_from_indices(top_x_indices)  # returns list of base64 strings

    messages = [
        {"role": "system", "content": "You are a visual concept tagger for neuroscience research."},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                *[
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img[0]}"}}
                    for img in encoded_images
                ]
            ]
        }
    ]

    try:
        response = client.chat.completions.create(
            model="gpt-4o",#"azure/gpt-4o",
            messages=messages,
            timeout=60
        )
        return response.choices[0].message.content.strip()

    except openai.OpenAIError as e:
        print(f"OpenAI API error: {e}")
        return None


In [ ]:
for layer in LAYERS:

    activation_path =f'/cosy/LLM_activations/activations/val_resnet18-layer{layer}.pt'

    # Load the activations 
    print(f"Loading activations from: {activation_path}")
    activations = torch.load(activation_path)
    print("Activation shape:", activations.shape)

    top_x_indices = retrieve_top_x_indices(activations, image_number)
    print("Top-X indices shape:", top_x_indices.shape)

    # take 50 random neurons: create a random sample of 50 indices and then take the corresponding neurons
    random_indices = np.random.choice(top_x_indices.shape[0], size=50, replace=False)
    top_50 = top_x_indices[random_indices]
    print("Random sample shape:", top_x_indices.shape)

    for image_number in IMAGE_NUMBER:

        start_time = time.time()

        responses = []
        for i, image_indices in enumerate(top_50):    

            neuron_indice = random_indices[i]

            print(f"Prompting neuron {neuron_indice + 1}...")
            concept = prompt_single(top_x_indices=image_indices.tolist(), prompt=list(prompts.values())[0])
            responses.append(concept)

            time.sleep(10)

        print("Number of responses:", len(responses))

        csv_file_path = f"experiments/prompts_1/layer_{layer}_{image_number}.csv"

        # Prepare the data for writing
        data_to_write = []
        for unit, neuron_indices in enumerate(top_50):
            concept = responses[unit] if unit < len(responses) else None
            neuron = random_indices[unit]
            data_to_write.append([layer, neuron, concept])

        # Write to the CSV file
        with open(csv_file_path, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["layer", "unit", "description"])  # Write header
            writer.writerows(data_to_write)

        print(f"Data successfully written to {csv_file_path}")

prompt_file_path = f"experiments/prompts_1/prompt.txt"
with open(prompt_file_path, "w") as prompt_file:
    prompt_file.write(prompts['0'])
print(f"Prompt successfully written to {prompt_file_path}")


Loading activations from: /Users/sinievanderben/Documents/ETH/tech/cosy/LLM_activations/activations/val_resnet18-layer1.pt
Activation shape: torch.Size([50000, 64])
Top-X indices shape: torch.Size([64, 1])
Random sample shape: torch.Size([64, 1])
Prompting neuron 56...
Prompting neuron 25...
Prompting neuron 20...
Prompting neuron 46...
Prompting neuron 9...
Prompting neuron 37...
Prompting neuron 55...
Prompting neuron 61...
Prompting neuron 29...
Prompting neuron 8...
Prompting neuron 60...
Prompting neuron 11...
Prompting neuron 49...
Prompting neuron 33...
Prompting neuron 31...
Prompting neuron 34...
Prompting neuron 3...
Prompting neuron 53...
Prompting neuron 50...
Prompting neuron 27...
Prompting neuron 23...
Prompting neuron 26...
Prompting neuron 48...
Prompting neuron 38...
Prompting neuron 10...
Prompting neuron 6...
Prompting neuron 18...
Prompting neuron 17...
Prompting neuron 30...
Prompting neuron 22...
Prompting neuron 14...
Prompting neuron 35...
Prompting neuron 24..